---
# Conditional Statement Debug
---
This code was created for the MIDOSS project using Dept. of Ecology data that is under user-agreement and can't be shared more broadly.  As such, this code has been trimmed to just show different cases of conditional statements and the results in order to give an example of how python interprets [logical statement priorities](https://docs.python.org/3/reference/expressions.html#operator-precedence).   

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml

In [2]:
# User inputs
file_dir  = '/Users/rmueller/Data/MIDOSS/DeptOfEcology/'
file_name = 'MuellerTrans4-30-20.xlsx'

# This is the precision used to calculate oil type weights 
# Trial and error showed that a precision of 2 is neccessary for weights
# to sum to 1.0
precision = 2

# The following list includes facilities used in Casey's origin/destination analysis 
# with names matching the Dept. of Ecology (DOE) database.  
# For example, the shapefile "Maxum Petroleum - Harbor Island Terminal" is labeled as 'Maxum (Rainer Petroleum)'
# in the DOE database.  I use the Ecology language here and will need to translate to Shapefile speak 
facility_names = [ 'BP Cherry Point Refinery', 'Shell Puget Sound Refinery', 'Shell Oil LP Seattle Distribution Terminal',
                   'Maxum (Rainer Petroleum)', 'Tidewater Snake River Terminal','Nustar Energy Tacoma', 
                   'SeaPort Sound Terminal', 'Tesoro Vancouver Terminal','Phillips 66 Ferndale Refinery', 
                   'Phillips 66 Tacoma Terminal', 'Andeavor Anacortes Refinery (formerly Tesoro)',
                   'Tesoro Port Angeles Terminal','U.S. Oil & Refining','Naval Air Station Whidbey Island (NASWI)',
                   'NAVSUP Manchester', 'Alon Asphalt Company (Paramount Petroleum)', 'Kinder Morgan Liquids Terminal - Harbor Island',
                   'Nustar Energy Tacoma', 'Tesoro Pasco Terminal', 'REG Grays Harbor, LLC', 'Tidewater Vancouver Terminal',
                   'TLP Management Services LLC (TMS)']

# define vessel and oil types to include in .yaml file
vessel_types = ['atb', 'barge', 'tanker']
oil_types    = ['akns', 'bunker', 'dilbit', 'jet', 'diesel', 'gas', 'other']
data_types    = ['total_gallons', 'fraction_of_total', 'number_of_transfers']

In [3]:
# Import columns are: (G) Deliverer, (H) Receiver, (O) Region, (P) Product, 
#                     (Q) Quantity in Gallons, (R) Transfer Type (oiling, Cargo, or Other)', 
#                     (w) DelivererTypeDescription, (x) ReceiverTypeDescription 
df = pd.read_excel(f'{file_dir}{file_name}',sheet_name='Vessel Oil Transfer', 
                   usecols="G,H,P,Q,R,W,X")

df.TransferQtyInGallon = df.TransferQtyInGallon.astype(float).round(precision)

# housekeeping
df['Receiver'] = df['Receiver'].str.replace('US Oil Tacoma ', 'U.S. Oil & Refining')
df['Receiver'] = df['Receiver'].str.replace('TLP', 'TLP Management Services LLC (TMS)')
df['Deliverer'] = df['Deliverer'].str.replace('US Oil Tacoma ', 'U.S. Oil & Refining')
df['Deliverer'] = df['Deliverer'].str.replace('TLP', 'TLP Management Services LLC (TMS)')

## Create name lists for the DOE oil types used in our monte-carlo oil categories

In [4]:
# identify all names of oils in DOE database that are attributed to our oil types
oil_classification = {}
for oil in oil_types:
    oil_classification[oil] = []

[nrows,ncols] = df.shape
for row in range(nrows):
    if 'CRUDE' in df.Product[row] and df.Product[row] not in oil_classification['akns']:
        oil_classification['akns'].append(df.Product[row])
    elif 'BAKKEN' in df.Product[row] and df.Product[row] not in oil_classification['akns']:
        oil_classification['akns'].append(df.Product[row])
    elif 'BUNKER' in df.Product[row] and df.Product[row] not in oil_classification['bunker']:
        oil_classification['bunker'].append(df.Product[row])
    elif 'BITUMEN' in df.Product[row] and df.Product[row] not in oil_classification['dilbit']:
        oil_classification['dilbit'].append(df.Product[row])
    elif 'DIESEL' in df.Product[row] and df.Product[row] not in oil_classification['diesel']:
        oil_classification['diesel'].append(df.Product[row])
    elif 'GASOLINE' in df.Product[row] and df.Product[row] not in oil_classification['gas']:
        oil_classification['gas'].append(df.Product[row])
    elif 'JET' in df.Product[row] and df.Product[row] not in oil_classification['jet']:
        oil_classification['jet'].append(df.Product[row])
    elif ('CRUDE' not in df.Product[row] and
          'BAKKEN' not in df.Product[row] and
          'BUNKER' not in df.Product[row] and
          'BITUMEN' not in df.Product[row] and
          'DIESEL' not in df.Product[row] and
          'GASOLINE' not in df.Product[row] and
          'JET' not in df.Product[row] and
          df.Product[row] not in oil_classification['other']):
        oil_classification['other'].append(df.Product[row])

{'akns': ['CRUDE OIL', 'BAKKEN'],
 'bunker': ['BUNKER OIL/HFO'],
 'dilbit': ['BITUMEN OIL (DIL OR SYN)'],
 'jet': ['JET FUEL/KEROSENE'],
 'diesel': ['DIESEL/MARINE GAS OIL', 'BIODIESEL', 'DIESEL LOW SULPHUR (ULSD)'],
 'gas': ['GASOLINE'],
 'other': ['ETHANOL',
  'LUBE OIL/MOTOR OIL',
  'Cat Feed/VGO',
  'CUTTER STOCK',
  'OILY WATER MIXTURE',
  'WASTE OIL',
  'Z-OTHER',
  'HYDRAULIC OIL',
  'OILY WASTE',
  'USED OIL',
  'NAPTHA',
  'EDIBLE/VEGETABLE OIL',
  'DECANT OIL',
  'NONENE',
  'ASPHALT/CREOSEOTE']}

### Create a dataframe that combines imports and exports from our selected marine terminals so that we can query based on ship-type and yield the combined sum of imports and exports

In [5]:
import_export_df = df.loc[        
        (df.Receiver.isin(facility_names)) |
        (df.Deliverer.isin(facility_names)) &
        (df.TransferType == 'Cargo'),
    ]
print(f'Combined df shape: {import_export_df.shape}')

Combined df shape: (4391, 7)


### Tally up total volume transferred by ship type

In [6]:
# This is the total gallons transferred by ship type and summed over all oil types
total = {}
## Tally up exports by ship type
for vessel in vessel_types:
    if vessel == 'atb':
        type_description = ['TANK BARGE','TUGBOAT']
        atb_import_total = df.loc[
            (df.TransferType == 'Cargo') &
            (df.DelivererTypeDescription.isin(type_description)) &
            (df.Deliverer.str.contains('ITB') | df.Deliverer.str.contains('ATB')) &
            (df.Receiver.isin(facility_names)),
        ]
        
        atb_export_total = df.loc[
            (df.TransferType == 'Cargo') &
            (df.ReceiverTypeDescription.isin(type_description)) &
            (df.Receiver.str.contains('ITB') | df.Receiver.str.contains('ATB')) & 
            (df.Deliverer.isin(facility_names)),
        ]
   
        total['atb'] = (atb_import_total.TransferQtyInGallon.sum(axis=0).item() + 
                        atb_export_total.TransferQtyInGallon.sum(axis=0).item()
                       )

## Sort cargo transfers by ship types.  We have three categories: tanker, atb, barge. ATB is the only one shown here.
1. `atb`: Listed in DOE dataset as `TANK BARGE` or `TUGBOAT` TypeDescription and `ATB` or `ITB` as Deliverer/Receiver


## Tally up `total_gallons`, `number_of_transfers`, `fraction_of_total`, where:
- `total_gallons`: Total gallons transfered for the particular oil type
- `fraction_of_total`: (`total_gallons` by oil type) / (total gallons across oil types)
- `number_of_transfers`: Total transfers for the particular oil type

I use `import_export_df` to pre-select cargo transfers to/from our selected marine terminals.  I tried many things here to try to force creation of fractions that sum to 1.0, including:
1. multipling by 100*int(numerator), integer divide (//) int(denominator), then divide ratio by 100 to convert back to float, e.g.:
```
test = (100 * int(2))//int(3)
test/100
```
2. Using decimal_divide and setting precision (which only appears to do anything for precision <= 2), e.g.:
```
destination[facility][vessel][oil]['fraction_of_total'] = decimal_divide(
     int(destination[facility][vessel][oil]['total_gallons']), 
     int(tanker_import_total[facility]),
     precision
 ).round(precision)
```
3. Dividing two integers (the method I am sticking with as it's cleaner than (1) and just as effetive)

In [7]:
all_US = {} # this is the output dictionary
import_US = {}
export_US = {}
for vessel in vessel_types:
    all_US[vessel] = {}
    import_US[vessel] = {}
    export_US[vessel] = {}
    for oil in oil_types:
        all_US[vessel][oil] = {}
        import_US[vessel][oil] = {}
        export_US[vessel][oil] = {}
        for data in data_types:
            all_US[vessel][oil][data] = 0
            import_US[vessel][oil][data] = 0
            export_US[vessel][oil][data] = 0

### ATB imports and exports

This first example is the right way.  Debug examples follow.

In [10]:
vessel = 'atb'
type_description = ['TANK BARGE','TUGBOAT']

# Now estimat fractions of total by oil types
for oil in oil_types:

    import_US[vessel][oil]['total_gallons'] = df.loc[
            (df.TransferType == 'Cargo') &
            (df.DelivererTypeDescription.isin(type_description)) &
            (df.Deliverer.str.contains('ITB') | df.Deliverer.str.contains('ATB')) &
            (df.Receiver.isin(facility_names)) &
            (df.Product.isin(oil_classification[oil])),
            'TransferQtyInGallon'
        ]
            
    export_US[vessel][oil]['total_gallons'] = df.loc[
            (df.TransferType == 'Cargo') &
            (df.ReceiverTypeDescription.isin(type_description)) &
            (df.Receiver.str.contains('ITB') | df.Receiver.str.contains('ATB')) & 
            (df.Deliverer.isin(facility_names)) &
            (df.Product.isin(oil_classification[oil])),
            'TransferQtyInGallon'
        ]

    # DEBUG: Check that the total number of records for 
    #  different oil types are the same in the two dictionaries 
    # used in the fraction calculation 
    numerator_total = (import_US[vessel][oil]['total_gallons'].shape[0] +
                       export_US[vessel][oil]['total_gallons'].shape[0])
    denominator_total = (atb_import_total.loc[atb_import_total.Product.isin(oil_classification[oil])].shape[0] +
                        atb_export_total.loc[atb_export_total.Product.isin(oil_classification[oil])].shape[0]
                        )

    print(f'number of transfers in numerator for {oil}: {numerator_total}')
    print(f'number of transfers in denominator for {oil}: {denominator_total}')
    # Add up net oil quantity transferred by oil_types
    all_US[vessel][oil]['total_gallons'] = (
        import_US[vessel][oil]['total_gallons'].sum(axis=0).item() + 
        export_US[vessel][oil]['total_gallons'].sum(axis=0).item()
    )

    # Calculate the fraction of total oil transfer represented 
    # by this oil type    
    if total[vessel] != 0:
        all_US[vessel][oil]['fraction_of_total'] = (
            int(all_US[vessel][oil]['total_gallons'])/
            int(total[vessel])
        )       

    # Catalogue the number of transfers 
    all_US[vessel][oil]['number_of_transfers'] = (
        import_US[vessel][oil]['total_gallons'].shape[0] + 
        export_US[vessel][oil]['total_gallons'].shape[0]
    ) 

number of transfers in numerator for akns: 6
number of transfers in denominator for akns: 6
number of transfers in numerator for bunker: 91
number of transfers in denominator for bunker: 91
number of transfers in numerator for dilbit: 0
number of transfers in denominator for dilbit: 0
number of transfers in numerator for jet: 72
number of transfers in denominator for jet: 72
number of transfers in numerator for diesel: 174
number of transfers in denominator for diesel: 174
number of transfers in numerator for gas: 236
number of transfers in denominator for gas: 236
number of transfers in numerator for other: 83
number of transfers in denominator for other: 83


---

BEGIN TESTING: What is the calculation if I remove ITB and ATB query entirely?

In [11]:
# Now estimat fractions of total by oil types
for oil in oil_types:

    import_US[vessel][oil]['total_gallons'] = df.loc[
            (df.TransferType == 'Cargo') &
            (df.DelivererTypeDescription.isin(type_description)) &
            (df.Receiver.isin(facility_names)) &
            (df.Product.isin(oil_classification[oil])),
            'TransferQtyInGallon'
        ]
            
    export_US[vessel][oil]['total_gallons'] = df.loc[
            (df.TransferType == 'Cargo') &
            (df.ReceiverTypeDescription.isin(type_description)) &
            (df.Deliverer.isin(facility_names)) &
            (df.Product.isin(oil_classification[oil])),
            'TransferQtyInGallon'
        ] 
    
    # DEBUG: Check that the total number of records for 
    #  different oil types are the same in the two dictionaries 
    # used in the fraction calculation 
    numerator_total = (import_US[vessel][oil]['total_gallons'].shape[0] +
                       export_US[vessel][oil]['total_gallons'].shape[0])
    denominator_total = (atb_import_total.loc[atb_import_total.Product.isin(oil_classification[oil])].shape[0] +
                        atb_export_total.loc[atb_export_total.Product.isin(oil_classification[oil])].shape[0]
                        )

    print(f'number of transfers in numerator for {oil}: {numerator_total}')
    print(f'number of transfers in denominator for {oil}: {denominator_total}')
    # Add up net oil quantity transferred by oil_types
    all_US[vessel][oil]['total_gallons'] = (
        import_US[vessel][oil]['total_gallons'].sum(axis=0).item() + 
        export_US[vessel][oil]['total_gallons'].sum(axis=0).item()
    )

    # Calculate the fraction of total oil transfer represented 
    # by this oil type    
    if total[vessel] != 0:
        all_US[vessel][oil]['fraction_of_total'] = (
            int(all_US[vessel][oil]['total_gallons'])/
            int(total[vessel])
        )       

    # Catalogue the number of transfers 
    all_US[vessel][oil]['number_of_transfers'] = (
        import_US[vessel][oil]['total_gallons'].shape[0] + 
        export_US[vessel][oil]['total_gallons'].shape[0]
    ) 

number of transfers in numerator for akns: 23
number of transfers in denominator for akns: 6
number of transfers in numerator for bunker: 1235
number of transfers in denominator for bunker: 91
number of transfers in numerator for dilbit: 6
number of transfers in denominator for dilbit: 0
number of transfers in numerator for jet: 255
number of transfers in denominator for jet: 72
number of transfers in numerator for diesel: 920
number of transfers in denominator for diesel: 174
number of transfers in numerator for gas: 606
number of transfers in denominator for gas: 236
number of transfers in numerator for other: 313
number of transfers in denominator for other: 83


This is the case of all barges in numerator vs. ATBs in denominator, and it makes sense that the numberator has consistently higher values than the denominator 

---

Now compare with incorrect grouping of | statement.

---

In [12]:
# Now estimate fractions of total by oil types
for oil in oil_types:

    import_US[vessel][oil]['total_gallons'] = df.loc[
            (df.TransferType == 'Cargo') &
            (df.DelivererTypeDescription.isin(type_description)) &
            (df.Deliverer.str.contains('ITB')) | (df.Deliverer.str.contains('ATB')) &
            (df.Receiver.isin(facility_names)) &
            (df.Product.isin(oil_classification[oil])),
            'TransferQtyInGallon'
        ]
            
    export_US[vessel][oil]['total_gallons'] = df.loc[
            (df.TransferType == 'Cargo') &
            (df.ReceiverTypeDescription.isin(type_description)) &
            (df.Deliverer.str.contains('ITB')) | (df.Deliverer.str.contains('ATB')) &
            (df.Deliverer.isin(facility_names)) &
            (df.Product.isin(oil_classification[oil])),
            'TransferQtyInGallon'
        ] 
    
    # DEBUG: Check that the total number of records for 
    #  different oil types are the same in the two dictionaries 
    # used in the fraction calculation 
    numerator_total = (import_US[vessel][oil]['total_gallons'].shape[0] +
                       export_US[vessel][oil]['total_gallons'].shape[0])
    denominator_total = (atb_import_total.loc[atb_import_total.Product.isin(oil_classification[oil])].shape[0] +
                        atb_export_total.loc[atb_export_total.Product.isin(oil_classification[oil])].shape[0]
                        )

    print(f'number of transfers in numerator for {oil}: {numerator_total}')
    print(f'number of transfers in denominator for {oil}: {denominator_total}')
    # Add up net oil quantity transferred by oil_types
    all_US[vessel][oil]['total_gallons'] = (
        import_US[vessel][oil]['total_gallons'].sum(axis=0).item() + 
        export_US[vessel][oil]['total_gallons'].sum(axis=0).item()
    )

    # Calculate the fraction of total oil transfer represented 
    # by this oil type    
    if total[vessel] != 0:
        all_US[vessel][oil]['fraction_of_total'] = (
            int(all_US[vessel][oil]['total_gallons'])/
            int(total[vessel])
        )       

    # Catalogue the number of transfers 
    all_US[vessel][oil]['number_of_transfers'] = (
        import_US[vessel][oil]['total_gallons'].shape[0] + 
        export_US[vessel][oil]['total_gallons'].shape[0]
    ) 

number of transfers in numerator for akns: 30
number of transfers in denominator for akns: 6
number of transfers in numerator for bunker: 67
number of transfers in denominator for bunker: 91
number of transfers in numerator for dilbit: 27
number of transfers in denominator for dilbit: 0
number of transfers in numerator for jet: 29
number of transfers in denominator for jet: 72
number of transfers in numerator for diesel: 56
number of transfers in denominator for diesel: 174
number of transfers in numerator for gas: 71
number of transfers in denominator for gas: 236
number of transfers in numerator for other: 86
number of transfers in denominator for other: 83


The results here are inconsistent.  For some oil types, the denominator is greater (i.e. more ATBs than non-ATBs); for some oil types the demonimator is less (i.e. fewer ATBs than non-ATBs).  

In [13]:
# Now estimate fractions of total by oil types
for oil in oil_types:

    import_US[vessel][oil]['total_gallons'] = df.loc[
            ((df.TransferType == 'Cargo') &
            (df.DelivererTypeDescription.isin(type_description)) &
            (df.Deliverer.str.contains('ITB'))) | ((df.Deliverer.str.contains('ATB')) &
            (df.Receiver.isin(facility_names)) &
            (df.Product.isin(oil_classification[oil]))),
            'TransferQtyInGallon'
        ]
            
    export_US[vessel][oil]['total_gallons'] = df.loc[
            ((df.TransferType == 'Cargo') &
            (df.ReceiverTypeDescription.isin(type_description)) &
            (df.Deliverer.str.contains('ITB'))) | ((df.Deliverer.str.contains('ATB')) &
            (df.Deliverer.isin(facility_names)) &
            (df.Product.isin(oil_classification[oil]))),
            'TransferQtyInGallon'
        ] 
    
    # DEBUG: Check that the total number of records for 
    #  different oil types are the same in the two dictionaries 
    # used in the fraction calculation 
    numerator_total = (import_US[vessel][oil]['total_gallons'].shape[0] +
                       export_US[vessel][oil]['total_gallons'].shape[0])
    denominator_total = (atb_import_total.loc[atb_import_total.Product.isin(oil_classification[oil])].shape[0] +
                        atb_export_total.loc[atb_export_total.Product.isin(oil_classification[oil])].shape[0]
                        )

    print(f'number of transfers in numerator for {oil}: {numerator_total}')
    print(f'number of transfers in denominator for {oil}: {denominator_total}')
    # Add up net oil quantity transferred by oil_types
    all_US[vessel][oil]['total_gallons'] = (
        import_US[vessel][oil]['total_gallons'].sum(axis=0).item() + 
        export_US[vessel][oil]['total_gallons'].sum(axis=0).item()
    )

    # Calculate the fraction of total oil transfer represented 
    # by this oil type    
    if total[vessel] != 0:
        all_US[vessel][oil]['fraction_of_total'] = (
            int(all_US[vessel][oil]['total_gallons'])/
            int(total[vessel])
        )       

    # Catalogue the number of transfers 
    all_US[vessel][oil]['number_of_transfers'] = (
        import_US[vessel][oil]['total_gallons'].shape[0] + 
        export_US[vessel][oil]['total_gallons'].shape[0]
    ) 

number of transfers in numerator for akns: 30
number of transfers in denominator for akns: 6
number of transfers in numerator for bunker: 67
number of transfers in denominator for bunker: 91
number of transfers in numerator for dilbit: 27
number of transfers in denominator for dilbit: 0
number of transfers in numerator for jet: 29
number of transfers in denominator for jet: 72
number of transfers in numerator for diesel: 56
number of transfers in denominator for diesel: 174
number of transfers in numerator for gas: 71
number of transfers in denominator for gas: 236
number of transfers in numerator for other: 86
number of transfers in denominator for other: 83


Take away: everything before the | statement is grouped together and everything after the | statement is grouped together as (before |) OR (after |); so the lesser priority statement effectively introduces brackets for how the conditions are evaluated and how the lesser priority statement is used.  

END TESTING

---
---